In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. Dataset class
class BrainTumorDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)

        # Normalize to [0, 1]
        img = img.astype("float32") / 255.0
        mask = mask.astype("float32") / 255.0
        mask = np.expand_dims(mask, axis=-1)

        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        else:
            img = torch.tensor(img).unsqueeze(0)
            mask = torch.tensor(mask).permute(2, 0, 1)

        return img, mask

# 2. Transformations
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2(),
])

# 3. Prepare data
image_dir = './data/dataset/images'
mask_dir = './data/dataset/masks'
all_images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
all_masks = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])

train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    all_images, all_masks, test_size=0.2, random_state=42
)

train_dataset = BrainTumorDataset(train_imgs, train_masks, transform=train_transform)
val_dataset = BrainTumorDataset(val_imgs, val_masks, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# 4. Model
model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet',in_channels=1, classes=1).cuda()

# 5. Loss & Optimizer
bce = nn.BCEWithLogitsLoss()

def dice_loss(pred, target, smooth=1.):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

def combined_loss(pred, target):
    return bce(pred, target) + dice_loss(pred, target)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 6. Training loop
for epoch in range(10):  # Increase for full training
    model.train()
    epoch_loss = 0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        imgs, masks = imgs.cuda(), masks.cuda()
        preds = model(imgs)
        loss = combined_loss(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_loader):.4f}")

    # Optionally add validation and saving model checkpoints



/home/luka/src/repos/BrainTumorDetection/.venv/lib/python3.12/site-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx